# Data Science en Entreprise

## API pôle emploi

Autorisation d’accès à l’api des offres d’emplois

Compte pole emploi :    
username : mohammed.didi@outlook.com    
password : Projet_Data1



In [1]:
import requests


def obtenir_token(client_id, client_secret):
    """
    Cette fonction obtient un token d'accès pour l'API de Pôle Emploi.
    """
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire'
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'api_offresdemploiv2 o2dsoffre'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.post(url_token, data=auth_data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Erreur lors de la requête : {response.text}")
    else:
        data = response.json()
        return data['access_token']
    
# Utilisation de la fonction
client_id = 'PAR_projetdata_29c785675807663802eabdf57858fc4e2ecd4eba4de8d05eee293bfdcacefae5'
client_secret = '8a48c4726bb2d33cafb7a636703a3b0ad70e4098b525063d2a99387a95a76b70'


In [24]:
import requests
from datetime import datetime
import pandas as pd
import json


def rechercher_offres(token, mot_cle, mois, annee, nb_offres=10):
    """
    Recherche les offres d'emploi dans le domaine des RH pour un mois et une année spécifiques.
    """
    url_offres = 'https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search'
    headers = {'Authorization': f'Bearer {token}'}

    # Calculer les dates de début et de fin du mois de novembre
    debut_mois = f"{annee}-{mois:02d}-01T00:00:00Z"
    fin_mois = f"{annee}-{mois:02d}-30T00:00:00Z"  # Novembre a 30 jours

    params = {
        'motsCles': mot_cle,  # Mots-clés pour le domaine des Ressources Humaines
        'range': f'0-{nb_offres-1}',  # Les 10 premières offres
        'minCreationDate': debut_mois,
        'maxCreationDate': fin_mois
    }

    response = requests.get(url_offres, headers=headers, params=params)

    try:
        return response.json()
    except:
        raise Exception(f"Erreur lors de la requête : {response.json()}")



token = obtenir_token(client_id, client_secret)
offres_Data_novembre = rechercher_offres(token, 'Data', 11, 2023)  # Exemple pour novembre 2023
# json to dataframe
# jsoon to dataframe
df = pd.DataFrame.from_dict(offres_Data_novembre['resultats'])


In [37]:
from datetime import datetime, timedelta

def get_access_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"
    client_id = "PAR_projetdata_20dc94f3568457527fe4edda39e827c960f36708a61e6bc3b9e0bdd5c798091d"
    client_secret = "58278acf29c5d865bdc657b658ec037342ba4c889459d16c09045b5b5437986f"
    scope = "api_offresdemploiv2 application_PAR_projetdata_20dc94f3568457527fe4edda39e827c960f36708a61e6bc3b9e0bdd5c798091d o2dsoffre"
    grant_type = "client_credentials"

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": scope,
        "grant_type": grant_type,
    }

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

def get_job_offers(access_token, min_creation_date, max_creation_date):
    url = "https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search"
    params = {
        "minCreationDate": min_creation_date,
        "maxCreationDate": max_creation_date
    }
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }

    response = requests.get(url, headers=headers, params=params)
    return response.json()


# Création du DataFrame vide
df = pd.DataFrame()

current_date = datetime.now()

# Boucle pour les 100 derniers jours
for day in range(1, 100):
    # Calculer la date il y a 'day' jours
    target_date = current_date - timedelta(days=day)
    
    # Formatter la date dans le format attendu
    min_date = target_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    max_date = (target_date + timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")

    # Obtenez le token d'accès
    access_token = get_access_token()

    # Récupérez les offres d'emploi
    job_offers = get_job_offers(access_token, min_date, max_date)

    # Ajoutez les offres d'emploi au DataFrame
    df = pd.concat([df, pd.json_normalize(job_offers["resultats"])])

# Réinitialisez l'index du DataFrame
df.reset_index(drop=True, inplace=True)

# Affichez le DataFrame final
print(df)

            id                                           intitule  \
0      167NDVL                      Ouvrier / Ouvrière paysagiste   
1      167NDVK  Conducteur / Conductrice de travaux en électri...   
2      167NDVJ  Conseiller(ère) Bien-Être et Médecines Douces ...   
3      167NDVH    Enquêteur Terrain - Visite en Supermarché (H/F)   
4      167NDVG                                    Maçon / Maçonne   
...        ...                                                ...   
14433  3541232                           Boucher / Bouchère (H/F)   
14434  3541005                       Boulanger / Boulangère (H/F)   
14435  3540910                                Chef d'équipe (H/F)   
14436  3540839             Serveur / Serveuse de restaurant (H/F)   
14437  3540732                               Réceptionnaire (H/F)   

                                             description  \
0      Vous n'aimez pas la routine, vous aimez vous i...   
1      VOTRE MISSION\nNous sommes à la recherche d'

In [38]:
df

,id,intitule,description,dateCreation,dateActualisation,romeCode,romeLibelle,appellationlibelle,typeContrat,typeContratLibelle,...,contact.urlPostulation,contact.coordonnees2,contact.coordonnees3,agence.courriel,conditionExercice,langues,complementExercice,contact.telephone,agence.telephone,origineOffre.partenaires
0,167NDVL,Ouvrier / Ouvrière paysagiste,"Vous n'aimez pas la routine, vous aimez vous i...",2024-01-11T10:44:23.000Z,2024-01-11T10:44:23.000Z,A1203,Aménagement et entretien des espaces verts,Ouvrier / Ouvrière paysagiste,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,167NDVK,Conducteur / Conductrice de travaux en électri...,VOTRE MISSION\nNous sommes à la recherche d'un...,2024-01-11T10:44:22.000Z,2024-01-11T10:44:23.000Z,F1201,Conducteur / Conductrice de travaux,Conducteur / Conductrice de travaux en électri...,MIS,Mission intérimaire - 6 Mois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,167NDVJ,Conseiller(ère) Bien-Être et Médecines Douces ...,En tant que Conseiller(ère) Naturellement Bien...,2024-01-11T10:44:21.000Z,2024-01-11T10:44:22.000Z,D1403,Relation commerciale auprès de particuliers,Conseiller vendeur / Conseillère vendeuse à do...,CCE,Profession commerciale,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,167NDVH,Enquêteur Terrain - Visite en Supermarché (H/F),Vous avez un sens affûté pour les détails et u...,2024-01-11T10:44:19.000Z,2024-01-11T10:44:21.000Z,M1401,Conduite d'enquêtes,Enquêteur / Enquêtrice terrain,CDD,Contrat à durée déterminée - 1 Jour(s),...,https://www.topo-marketing.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167NDVG,Maçon / Maçonne,Allia Intérim recrute ! \n\nNous recherchons p...,2024-01-11T10:44:18.000Z,2024-01-11T10:44:20.000Z,F1703,Maçonnerie,Maçon / Maçonne,MIS,Mission intérimaire - 5 Mois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14433,3541232,Boucher / Bouchère (H/F),Description du poste :\nSous le contrôle du ch...,2023-10-04T16:30:30.000Z,2024-01-04T09:56:38.000Z,D1101,Boucherie,Boucher / Bouchère,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'nom': 'METEOJOB', 'url': 'https://www.meteo..."
14434,3541005,Boulanger / Boulangère (H/F),Description du poste :\nAu sein de notre labor...,2023-10-04T16:29:10.000Z,2024-01-04T12:16:25.000Z,D1102,Boulangerie - viennoiserie,Boulanger / Boulangère,CDD,Contrat à durée déterminée - 3 Mois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'nom': 'METEOJOB', 'url': 'https://www.meteo..."
14435,3540910,Chef d'équipe (H/F),Description du poste :\nDESCRIPTIF DU POSTE\nL...,2023-10-04T16:28:30.000Z,2023-12-19T14:04:13.000Z,A1203,Aménagement et entretien des espaces verts,Chef d'équipe paysagiste,CDI,Contrat à durée indéterminée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'nom': 'METEOJOB', 'url': 'https://www.meteo..."
14436,3540839,Serveur / Serveuse de restaurant (H/F),Description du poste :\nDynamisme et sens du c...,2023-10-04T16:28:08.000Z,2024-01-04T12:19:19.000Z,G1803,Service en restauration,Serveur / Serveuse de restaurant,MIS,Mission intérimaire - 90 Jour(s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'nom': 'METEOJOB', 'url': 'https://www.meteo..."


In [39]:
df.columns

Index(['id', 'intitule', 'description', 'dateCreation', 'dateActualisation',
       'romeCode', 'romeLibelle', 'appellationlibelle', 'typeContrat',
       'typeContratLibelle', 'natureContrat', 'experienceExige',
       'experienceLibelle', 'experienceCommentaire', 'formations', 'permis',
       'competences', 'dureeTravailLibelle', 'dureeTravailLibelleConverti',
       'alternance', 'nombrePostes', 'accessibleTH', 'deplacementCode',
       'deplacementLibelle', 'qualificationCode', 'qualificationLibelle',
       'codeNAF', 'secteurActivite', 'secteurActiviteLibelle',
       'lieuTravail.libelle', 'lieuTravail.latitude', 'lieuTravail.longitude',
       'lieuTravail.codePostal', 'lieuTravail.commune', 'entreprise.nom',
       'entreprise.entrepriseAdaptee', 'salaire.commentaire',
       'salaire.complement1', 'contact.nom', 'contact.coordonnees1',
       'contact.courriel', 'origineOffre.origine', 'origineOffre.urlOrigine',
       'qualitesProfessionnelles', 'offresManqueCandidats', 'sa

In [40]:
# save to csv in ../data

df.to_csv('../data/offres_last99.csv', index=False)
